In [1]:
# warnings kütüphanesi uyarıları kapatmak için kullanılır
import warnings 
warnings.filterwarnings("ignore")

In [2]:
# GPU ve CUDA sürümlerini kontrol etmek için kullanılır
!nvidia-smi

Wed Jun 26 14:37:51 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [5]:
from datasets import load_dataset

from transformers import GPT2LMHeadModel, GPT2Tokenizer
from transformers import TrainingArguments, Trainer

import torch 

In [1]:
print("datasets version: 2.19.2")
print("transformers version: 4.41.2")
print("torch version: 2.1.2")

datasets version: 2.19.2
transformers version: 4.41.2
torch version: 2.1.2


In [6]:
from huggingface_hub import notebook_login

## Huggingface token, https://huggingface.co/settings/profile
notebook_login()

## 🤗 Hugging Face Datasets Kütüphanesi

### Parametreler

- **`path`**:
  - Yüklenecek veri setinin yolu veya ismi. Örnek: "imdb", "glue".

- **`name`**:
  - Yüklenecek veri setinin alt kümesi. Örnek: "sst2" (GLUE için).

- **`data_dir`**:
  - Veri dosyalarının bulunduğu dizin.

- **`data_files`**:
  - Yüklenecek veri dosyaları.

- **`split`**:
  - Veri setinin bölünmesi (örneğin "train", "test").

- **`cache_dir`**:
  - Verinin önbelleğe alınacağı dizin.

- **`features`**:
  - Özelliklerin açıkça belirtildiği yer.

- **`download_config`**:
  - İndirme yapılandırma ayarları.

- **`download_mode`**:
  - İndirme modu: "reuse_dataset_if_exists", "reuse_cache_if_exists", "force_redownload".

- **`verification_mode`**:
  - Veri setinin doğrulama modu.

- **`ignore_verifications`**:
  - Artık kullanılmıyor, doğrulamaları atlamak için.

- **`keep_in_memory`**:
  - Eğer True ise, veri seti bellekte tutulur.

- **`save_infos`**:
  - Eğer True ise, veri seti bilgileri kaydedilir.

- **`revision`**:
  - Yüklenecek veri setinin versiyonu veya commit ID'si.

- **`token`**:
  - Private veri setleri için kullanılır, bir token sağlar.

- **`use_auth_token`**:
  - Artık kullanılmıyor, oturum açma token'ı için.

- **`task`**:
  - Artık kullanılmıyor, veri seti yükleme görevini belirtmek için.

- **`streaming`**:
  - Eğer True ise, veri seti akış modunda yüklenir.

- **`num_proc`**:
  - Çok işlemcili veri işleme için işlemci sayısı.

- **`storage_options`**:
  - Depolama seçenekleri.

- **`trust_remote_code`**:
  - Eğer True ise, uzaktan kod çalıştırmaya izin verir.

- **`**config_kwargs`**:
  - Diğer ek yapılandırma argümanları.


In [7]:
dataset = load_dataset('Trelis/tiny-shakespeare', split='train')

Generating train split:   0%|          | 0/472 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/49 [00:00<?, ? examples/s]

In [8]:
print(dataset)
print("======")
print(dataset[0])

Dataset({

    features: ['Text'],

    num_rows: 472

})


{'Text': "First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you know Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us kill him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be done: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citizens, the patricians good.\nWhat authority surfeits on would relieve us: if they\nwould yield us but the superfluity, while it were\nwholesome, we might guess they relieved us humanely;\nbut they think we are too dear: the leanness that\nafflicts us, the object of our misery, is as an\ninventory to particularise their abundance; our\nsufferance is a gain to them Let us revenge this with\nour p

## 🤗 Hugging Face Tokenizer Kütüphanesi

### Parametreler

- **`text`**:
  - Tokenize edilecek metin veya metinlerin listesi.

- **`text_pair`**:
  - İkinci bir metin veya metinlerin listesi, çift metinli modeller için kullanılır.

- **`text_target`**:
  - Hedef metin veya metinlerin listesi (genellikle seq2seq modelleri için kullanılır).

- **`text_pair_target`**:
  - Hedef ikinci metin veya metinlerin listesi, çift metinli modeller için kullanılır.

- **`add_special_tokens`**:
  - Özel tokenler ekler (örneğin [CLS], [SEP]). True olması, modeli daha iyi performans gösterir.

- **`padding`**:
  - Padding stratejisi: True, False, "longest", "max_length".

- **`truncation`**:
  - Kesme stratejisi: True, False, "longest_first", "only_first".

- **`max_length`**:
  - Maksimum token sayısı. Yüksek değerler daha fazla bilgi taşır ama daha fazla bellek kullanır.

- **`stride`**:
  - Kesme sırasında kayma boyutu. Uzun metinler için daha küçük değerler kullanışlı olabilir.

- **`is_split_into_words`**:
  - Eğer True ise, metin kelimelere bölünmüş olarak kabul edilir.

- **`pad_to_multiple_of`**:
  - Padding boyutunun bir katı olacak şekilde padding ekler.

- **`return_tensors`**:
  - Döndürülecek tensör tipi: 'pt', 'tf', 'np'.

- **`return_token_type_ids`**:
  - Eğer True ise, token tipi ID'lerini döndürür.

- **`return_attention_mask`**:
  - Eğer True ise, attention maskelerini döndürür.

- **`return_overflowing_tokens`**:
  - Eğer True ise, taşan tokenleri döndürür.

- **`return_special_tokens_mask`**:
  - Eğer True ise, özel token maskesini döndürür.

- **`return_offsets_mapping`**:
  - Eğer True ise, offset mapping döndürür.

- **`return_length`**:
  - Eğer True ise, token uzunluklarını döndürür.

- **`verbose`**:
  - Eğer True ise, işlem hakkında daha fazla bilgi verir.

- **`**kwargs`**:
  - Diğer ek argümanlar.


In [2]:
base_model_name = "openai-community/gpt2"

tokenizer = GPT2Tokenizer.from_pretrained(base_model_name)

tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

c:\Users\90530\anaconda3\lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\90530\.cache\huggingface\hub\models--openai-community--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.

To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development

  warnings.warn(message)


vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [10]:
print(tokenizer)

GPT2Tokenizer(name_or_path='openai-community/gpt2', vocab_size=50257, model_max_length=1024, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '<|endoftext|>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={

	50256: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),

}


In [11]:
def text_tokenizer(text):
    tokenized_output = tokenizer(
        text['Text'], 
        truncation=True,
        padding='max_length', 
        max_length=128
    )

    # labels input_ids ile aynı olacak
    tokenized_output['labels'] = tokenized_output['input_ids'].copy()
    return tokenized_output

train_data = dataset.map(text_tokenizer, batched=True)

Map:   0%|          | 0/472 [00:00<?, ? examples/s]

In [12]:
print(train_data)
print("=====")
print(train_data[0])

Dataset({

    features: ['Text', 'input_ids', 'attention_mask', 'labels'],

    num_rows: 472

})

=====

{'Text': "First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you know Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us kill him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be done: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citizens, the patricians good.\nWhat authority surfeits on would relieve us: if they\nwould yield us but the superfluity, while it were\nwholesome, we might guess they relieved us humanely;\nbut they think we are too dear: the leanness that\nafflicts us, the object of our misery, is as an\ninventory to particularise their abundance; our\nsufferance is

In [13]:
model = GPT2LMHeadModel.from_pretrained(base_model_name)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # GPU varsa GPU kullanır, yoksa CPU kullanır

model.to(device) 

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

## 🤗 Hugging Face TrainingArguments Kütüphanesi

### Parametreler

- **`output_dir`**:
  - Eğitim sırasında ve sonrasında sonuçların kaydedileceği dizin.

- **`overwrite_output_dir`**:
  - Eğer True ise, mevcut `output_dir` içeriğini üzerine yazar.

- **`do_train`**:
  - Eğitim işleminin yapılıp yapılmayacağını belirtir.

- **`do_eval`**:
  - Değerlendirme işleminin yapılıp yapılmayacağını belirtir.

- **`do_predict`**:
  - Tahmin işleminin yapılıp yapılmayacağını belirtir.

- **`eval_strategy`**:
  - Değerlendirme stratejisi: 'no', 'steps' veya 'epoch'.

- **`prediction_loss_only`**:
  - Eğer True ise, sadece kayıp hesaplanır, tahminler döndürülmez.

- **`per_device_train_batch_size`**:
  - Her bir cihaz (GPU/CPU) için eğitim batch boyutu.

- **`per_device_eval_batch_size`**:
  - Her bir cihaz (GPU/CPU) için değerlendirme batch boyutu.

- **`gradient_accumulation_steps`**:
  - Gradient biriktirme adımları. Yüksek değerler bellek kullanımını azaltır ama eğitim süresini uzatır.

- **`eval_accumulation_steps`**:
  - Değerlendirme biriktirme adımları. Yüksek değerler bellek kullanımını azaltır ama değerlendirme süresini uzatır.

- **`eval_delay`**:
  - Eğitim başladıktan sonra ilk değerlendirmenin yapılacağı adım sayısı.

- **`learning_rate`**:
  - Öğrenme oranı. Yüksek değerler daha hızlı öğrenir ama aşırı öğrenmeye (overfitting) yol açabilir.

- **`weight_decay`**:
  - Ağırlıkların küçülme oranı. Yüksek değerler modelin genelleme yeteneğini artırabilir ama öğrenme yavaşlar.

- **`adam_beta1`**:
  - Adam optimizasyon algoritması için beta1 parametresi.

- **`adam_beta2`**:
  - Adam optimizasyon algoritması için beta2 parametresi.

- **`adam_epsilon`**:
  - Adam optimizasyon algoritması için epsilon parametresi.

- **`max_grad_norm`**:
  - Gradientlerin maksimum normu. Büyük değerler modelin stabilitesini artırabilir.

- **`num_train_epochs`**:
  - Eğitim için epoch sayısı. Yüksek değerler daha fazla öğrenme sağlar ama aşırı öğrenmeye yol açabilir.

- **`max_steps`**:
  - Maksimum eğitim adım sayısı. -1 ise, tüm epoch'lar tamamlanır.

- **`lr_scheduler_type`**:
  - Öğrenme oranı zamanlayıcı tipi.

- **`lr_scheduler_kwargs`**:
  - Öğrenme oranı zamanlayıcısı için ek parametreler.

- **`warmup_ratio`**:
  - Öğrenme oranı ısınma oranı. Yüksek değerler başlangıçta daha yavaş öğrenme sağlar.

- **`warmup_steps`**:
  - Öğrenme oranı ısınma adım sayısı. Yüksek değerler başlangıçta daha yavaş öğrenme sağlar.

- **`log_level`**:
  - Log seviyesi. 'passive', 'info', 'warning', 'error' veya 'critical'.

- **`log_level_replica`**:
  - Çoklu GPU eğitiminde log seviyesi.

- **`log_on_each_node`**:
  - Çoklu node eğitiminde her node için loglama yapılır.

- **`logging_dir`**:
  - TensorBoard logları için dizin.

- **`logging_strategy`**:
  - Loglama stratejisi: 'no', 'steps' veya 'epoch'.

- **`logging_first_step`**:
  - Eğer True ise, ilk adımda loglama yapılır.

- **`logging_steps`**:
  - Kaç adımda bir loglama yapılacağı.

- **`logging_nan_inf_filter`**:
  - Eğer True ise, NaN ve sonsuz değerler loglanmaz.

- **`save_strategy`**:
  - Modelin kaydedilme stratejisi: 'no', 'steps' veya 'epoch'.

- **`save_steps`**:
  - Kaç adımda bir modelin kaydedileceği.

- **`save_total_limit`**:
  - Maksimum kaç model kaydedileceği.

- **`save_safetensors`**:
  - Modelin güvenli tensör formatında kaydedilip kaydedilmeyeceği.

- **`save_on_each_node`**:
  - Eğer True ise, her node'da model kaydedilir.

- **`save_only_model`**:
  - Eğer True ise, sadece model kaydedilir, optimizer ve lr scheduler kaydedilmez.

- **`restore_callback_states_from_checkpoint`**:
  - Callback durumlarının checkpoint'ten geri yüklenip yüklenmeyeceği.

- **`no_cuda`**:
  - Eğer True ise, GPU kullanılmaz.

- **`use_cpu`**:
  - Eğer True ise, CPU kullanılır.

- **`use_mps_device`**:
  - Eğer True ise, MacOS Metal Performance Shaders kullanılır.

- **`seed`**:
  - Rastgelelik için seed değeri.

- **`data_seed`**:
  - Veri yükleme için seed değeri.

- **`jit_mode_eval`**:
  - Eğer True ise, PyTorch JIT modu kullanılır.

- **`use_ipex`**:
  - Eğer True ise, Intel Extension for PyTorch kullanılır.

- **`bf16`**:
  - Eğer True ise, bfloat16 kullanılır.

- **`fp16`**:
  - Eğer True ise, float16 kullanılır.

- **`fp16_opt_level`**:
  - float16 optimizasyon seviyesi.

- **`half_precision_backend`**:
  - Yarı hassasiyet arka ucu: 'auto', 'amp' veya 'apex'.

- **`bf16_full_eval`**:
  - Eğer True ise, bfloat16 tam değerlendirme yapılır.

- **`fp16_full_eval`**:
  - Eğer True ise, float16 tam değerlendirme yapılır.

- **`tf32`**:
  - Eğer True ise, TensorFloat-32 kullanılır.

- **`local_rank`**:
  - Çoklu GPU eğitiminde lokal rank.

- **`ddp_backend`**:
  - DDP backend tipi: 'nccl', 'gloo', 'mpi'.

- **`tpu_num_cores`**:
  - TPU çekirdek sayısı.

- **`tpu_metrics_debug`**:
  - TPU metriklerinin debug bilgilerini içerir.

- **`debug`**:
  - Debug seçeneği: 'underflow_overflow', 'poor_optimization'.

- **`dataloader_drop_last`**:
  - Eğer True ise, dataloader son batch'i düşürür.

- **`eval_steps`**:
  - Değerlendirme adım sayısı.

- **`dataloader_num_workers`**:
  - Dataloader için çalışan sayısı. Yüksek değerler hız artırır ama daha fazla bellek kullanır.

- **`dataloader_prefetch_factor`**:
  - Dataloader için prefetch faktörü. Yüksek değerler hız artırır ama daha fazla bellek kullanır.

- **`past_index`**:
  - Geçmiş index.

- **`run_name`**:
  - Çalışma ismi.

- **`disable_tqdm`**:
  - Eğer True ise, tqdm progress bar devre dışı bırakılır.

- **`remove_unused_columns`**:
  - Eğer True ise, kullanılmayan sütunlar veri setinden kaldırılır.

- **`label_names`**:
  - Label isimleri.

- **`load_best_model_at_end`**:
  - Eğitim sonunda en iyi modelin yüklenip yüklenmeyeceği.

- **`metric_for_best_model`**:
  - En iyi modelin seçilmesi için kullanılacak metrik.

- **`greater_is_better`**:
  - Eğer True ise, metriklerde yüksek değerler daha iyi olarak kabul edilir.

- **`ignore_data_skip`**:
  - Eğer True ise, veri atlamaları göz ardı edilir.

- **`fsdp`**:
  - Fully Sharded Data Parallel ayarları.

- **`fsdp_min_num_params`**:
  - Fully Sharded Data Parallel için minimum parametre sayısı.

- **`fsdp_config`**:
  - Fully Sharded Data Parallel için ek ayarlar.

- **`fsdp_transformer_layer_cls_to_wrap`**:
  - Fully Sharded Data Parallel için transformer layer sınıfı.

- **`accelerator_config`**:
  - Accelerator için ek ayarlar.

- **`deepspeed`**:
  - DeepSpeed yapılandırması.

- **`label_smoothing_factor`**:
  - Label smoothing faktörü.

- **`optim`**:
  - Optimizasyon algoritması.

- **`optim_args`**:
  - Optimizasyon algoritması için ek argümanlar.

- **`adafactor`**:
  - Eğer True ise, Adafactor optimizasyon algoritması kullanılır.

- **`group_by_length`**:
  - Eğer True ise, input uzunluğuna göre gruplanır.

- **`length_column_name`**:
  - Uzunluk sütunu ismi.

- **`report_to`**:
  - Raporlama platformları (örneğin 'wandb').

- **`ddp_find_unused_parameters`**:
  - DDP'de kullanılmayan parametrelerin bulunması.

- **`ddp_bucket_cap_mb`**:
  - DDP bucket kapasitesi.

- **`ddp_broadcast_buffers`**:
  - DDP broadcast buffer'ları.

- **`dataloader_pin_memory`**:
  - Dataloader için bellek pinleme.

- **`dataloader_persistent_workers`**:
  - Dataloader için kalıcı çalışanlar.

- **`skip_memory_metrics`**:
  - Eğer True ise, bellek metrikleri atlanır.

- **`use_legacy_prediction_loop`**:
  - Eğer True ise, eski tahmin döngüsü kullanılır.

- **`push_to_hub`**:
  - Eğer True ise, model Hugging Face Hub'a itilir.

- **`resume_from_checkpoint`**:
  - Checkpoint'ten devam edilir.

- **`hub_model_id`**:
  - Hugging Face Hub model ID.

- **`hub_strategy`**:
  - Hub stratejisi.

- **`hub_token`**:
  - Hugging Face Hub token.

- **`hub_private_repo`**:
  - Eğer True ise, özel repo kullanılır.

- **`hub_always_push`**:
  - Eğer True ise, her zaman Hub'a itilir.

- **`gradient_checkpointing`**:
  - Eğer True ise, gradient checkpointing yapılır.

- **`gradient_checkpointing_kwargs`**:
  - Gradient checkpointing için ek argümanlar.

- **`include_inputs_for_metrics`**:
  - Eğer True ise, değerlendirme metrikleri için inputlar da dahil edilir.

- **`eval_do_concat_batches`**:
  - Eğer True ise, değerlendirme batch'leri birleştirilir.

- **`fp16_backend`**:
  - float16 arka ucu.

- **`evaluation_strategy`**:
  - Değerlendirme stratejisi: 'no', 'steps' veya 'epoch'.

- **`push_to_hub_model_id`**:
  - Hub model ID.

- **`push_to_hub_organization`**:
  - Hub organizasyon ID.

- **`push_to_hub_token`**:
  - Hub token.

- **`mp_parameters`**:
  - Model paralel parametreler.

- **`auto_find_batch_size`**:
  - Eğer True ise, batch boyutu otomatik bulunur.

- **`full_determinism`**:
  - Eğer True ise, deterministik eğitim yapılır.

- **`torchdynamo`**:
  - TorchDynamo kullanımı.

- **`ray_scope`**:
  - Ray scope.

- **`ddp_timeout`**:
  - DDP timeout süresi.

- **`torch_compile`**:
  - Eğer True ise, Torch compile kullanılır.

- **`torch_compile_backend`**:
  - Torch compile backend.

- **`torch_compile_mode`**:
  - Torch compile modu.

- **`dispatch_batches`**:
  - Batch dispatching.

- **`split_batches`**:
  - Batch splitting.

- **`include_tokens_per_second`**:
  - Eğer True ise, saniye başına token sayısı dahil edilir.

- **`include_num_input_tokens_seen`**:
  - Eğer True ise, görülen input token sayısı dahil edilir.

- **`neftune_noise_alpha`**:
  - Neptune noise alpha.

- **`optim_target_modules`**:
  - Optimizasyon hedef modüller.

- **`batch_eval_metrics`**:
  - Eğer True ise, batch değerlendirme metrikleri hesaplanır.

In [18]:
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="no", # Validasyon verisi olmadığı için bu seçeneği kullanıyoruz
    learning_rate=2e-5,
    per_device_train_batch_size=1,  
    do_eval=False, # Validasyon verisi olmadığı için bu seçeneği kullanıyoruz
    num_train_epochs=10,
    weight_decay=0.01,
    push_to_hub=True, # Modeli huggingface hub'a yüklemek istediğimizi belirtiyoruz, False olarak ayarlanırsa model yüklenmez
    hub_model_id='gpt2-shakespeare-text-generation' # push_to_hub=True olduğunda modelin hub'da hangi isimle kaydedileceğini belirtir
)

## 🤗 Hugging Face Trainer Kütüphanesi

### Parametreler

- **`model`**:
  - Eğitim ve değerlendirme için kullanılacak model.

- **`args`**:
  - Eğitim argümanları, `TrainingArguments` sınıfı ile oluşturulur.

- **`data_collator`**:
  - Veriyi toplamak için kullanılan data collator.

- **`train_dataset`**:
  - Eğitim için kullanılacak veri seti.

- **`eval_dataset`**:
  - Değerlendirme için kullanılacak veri seti.

- **`tokenizer`**:
  - Metin verilerini modelin anlayabileceği sayısal değerlere dönüştürür.

- **`model_init`**:
  - Modelin başlatılması için kullanılacak fonksiyon.

- **`compute_metrics`**:
  - Değerlendirme metriklerini hesaplamak için kullanılacak fonksiyon.

- **`callbacks`**:
  - Eğitim sürecini izlemek ve kontrol etmek için kullanılacak callback listesi.

- **`optimizers`**:
  - Optimizasyon algoritması ve öğrenme oranı zamanlayıcısı.

- **`preprocess_logits_for_metrics`**:
  - Metrik hesaplama için logits'i ön işleme fonksiyonu.


In [19]:
trainer = Trainer(
    model=model, # Modelimiz
    args=training_args, # Training argümanlarımız
    train_dataset=train_data, # Eğitim verisetimiz
)

In [20]:
# Wandb-key ile wandb hesabınıza bağlanabilirsiniz
trainer.train()

Step,Training Loss
500,2.668800
1000,2.437100
1500,2.292200
2000,2.218700


TrainOutput(global_step=2360, training_loss=2.370756168688758, metrics={'train_runtime': 543.5846, 'train_samples_per_second': 8.683, 'train_steps_per_second': 4.342, 'total_flos': 308324597760000.0, 'train_loss': 2.370756168688758, 'epoch': 10.0})

In [21]:
trainer.push_to_hub() # Modeli huggingface hub'a yükler, push_to_hub=True olmalıdır

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

events.out.tfevents.1719350107.805e69c35453.34.1:   0%|          | 0.00/6.30k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/fawern/gpt2-shakespeare-text-generation/commit/e8d2097247aa2219d71356996c82fe1a79164443', commit_message='End of training', commit_description='', oid='e8d2097247aa2219d71356996c82fe1a79164443', pr_url=None, pr_revision=None, pr_num=None)

## 🤗 Hugging Face `model.generate` Kütüphanesi

### Parametreler

- **`inputs`**:
  - Girdi tensörü. Modelin kullanacağı başlangıç verileri.

- **`generation_config`**:
  - Opsiyonel jenerasyon konfigürasyonu. Jenerasyon süreci için yapılandırma ayarları.

- **`logits_processor`**:
  - Opsiyonel logits işlemcisi listesi. Logits değerlerini işlemek ve düzenlemek için kullanılır.

- **`stopping_criteria`**:
  - Opsiyonel durdurma kriterleri listesi. Jenerasyonun durması gereken durumları tanımlar.

- **`prefix_allowed_tokens_fn`**:
  - Opsiyonel bir fonksiyon. Jenerasyon sırasında izin verilen token'ları belirtir.

- **`synced_gpus`**:
  - Eğer True ise, çoklu GPU senkronizasyonu yapılır.

- **`assistant_model`**:
  - Opsiyonel yardımcı model. Jenerasyon sürecinde destekleyici model olarak kullanılır.

- **`streamer`**:
  - Opsiyonel bir streamer. Jenerasyon sonuçlarını anlık olarak aktarmak için kullanılır.

- **`negative_prompt_ids`**:
  - Opsiyonel negatif prompt ID'leri. Negatif örneklerin ID'lerini içerir.

- **`negative_prompt_attention_mask`**:
  - Opsiyonel negatif prompt dikkat maskesi. Negatif örneklerin dikkat maskesini tanımlar.

In [22]:
input_texts = [
    "To be, or not to be, that is the question:",
    "Two households, both alike in dignity, in fair Verona, where we lay our scene, from ancient grudge break to new mutiny",
    "All the world's a stage, and all the men and women merely players:"
]

trained_model = trainer.model

def generator(text):
    # Text dizisini modelin anlayabileceği forma çeviriyoruz
    input_ids = tokenizer.encode(text, return_tensors='pt').to(device)
    
    # Modeli kullanarak text üretimi yapıyoruz
    output = trained_model.generate(
        input_ids,
        max_length=100,   
    )
    
    # Üretilen texti decode ediyoruz
    generated_text = tokenizer.decode(
        output[0], 
        skip_special_tokens=True,
        num_return_sequences=1,
        no_repeat_ngram_size=2
    )
    
    return generated_text

In [23]:
print(generator(input_texts[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


To be, or not to be, that is the question:



First, I think, to be a matter of opinion.



Second, I think it best to be a man;

and therefore, as a man, as a man.



Third, I think it best to be a farmer;

and therefore, as a man.



I think it best to be a soldier;

and therefore, as a man.



I think it best to be


In [24]:
print(generator(input_texts[1]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Two households, both alike in dignity, in fair Verona, where we lay our scene, from ancient grudge break to new mutiny

and to the clamour of the new-married's blood,

The old and the new-married, the two kindreds,

The old and the new-married, the old and the old,

The two kindreds, the old and New-married,

The old and the old, the old and the old,

The old


In [25]:
print(generator(input_texts[2]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


All the world's a stage, and all the men and women merely players:

The best is to play the best,

And that's the best is to play the best.



GLOUCESTER:



PERDITA:

I' the best is to play the best,

And that's the best is to play the best.



GLOUCESTER:



PERDITA:

I' the best is to play the best,


